**Scrape Spotrac Data**

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.spotrac.com/nfl/buffalo-bills/cap/2021/')
soup = BeautifulSoup(driver.page_source,'lxml')
driver.quit()

table = soup.find_all('table')[0]  #This is the index of any table of that page. If you change it you can get different tables.
tab_data = [[celldata.text for celldata in rowdata.find_all(["th","td"])]
                        for rowdata in table.find_all("tr")]
for data in tab_data: #first and last rows are names of the columns and team totals respectively
    print(' '.join(data))


Active Players  (55) Pos. Base Salary Signing Bonus Roster Bonus Option Bonus Workout Bonus Restruc. Bonus Misc. Dead Cap Cap Hit Cap %
Dawkins
Dion Dawkins
 LT $7,300,000 $1,720,000 $235,294 $1,875,000 $250,000 - - ($21,930,000) $11,380,294  6.04
Allen
Josh Allen
 QB $920,000 $6,671,461 $2,618,596 - - - - ($103,410,057) $10,210,057  5.42
Hughes
Jerry Hughes
 DE $6,100,000 $2,100,000 $1,000,000 - $250,000 - - ($9,450,000) $9,450,000  5.02
Morse
Mitch Morse
 C $4,125,000 $2,750,000 $1,352,941 - $100,000 - - ($10,725,000) $8,327,941  4.42
Addison
Mario Addison
 DE $4,075,000 $2,000,000 $1,852,941 - $250,000 - - ($9,825,000) $8,177,941  4.34
Milano
Matt Milano
 OLB $5,500,000 $1,750,000 $294,118 - $250,000 - - ($20,250,000) $7,794,118  4.14
Lotulelei
Star Lotulelei
 DT $4,500,000 $2,600,000 $235,294 - $250,000 - - ($12,050,000) $7,585,294  4.03
Beasley
Cole Beasley
 WR $4,700,000 $1,500,000 $970,576 - $200,000 - - ($8,400,000) $7,370,576  3.91
Hyde
Micah Hyde
 SS $2,100,000 $4,100,000 $2